<a href="https://colab.research.google.com/github/Thanat-Music/table_to_dataframe/blob/main/3th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!sudo apt install tesseract-ocr-tha
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr-tha is already the newest version (4.00~git24-0e00fe6-1.2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [22]:
import cv2
import numpy as np
import pytesseract
from PIL import ImageFont, ImageDraw,Image
from google.colab.patches import cv2_imshow

In [33]:
def box_cut(values):
    row_img=[]
    dtype = [('x', int), ('y', int), ('w', int), ('h', int)]
    box_array = np.array(values, dtype = dtype)
    sort_box_array = np.sort(box_array, order = 'x')
    for position in sort_box_array:
        x, y, w, h = position
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
        #cv2_imshow(img)
        bg_img = np.zeros((img.shape[0], img.shape[1]))
        bg_img[y:y+h, x:x+w] = 1
        img_cut = img_copy[y:y+h, x:x+w,:]

        #cv2_imshow(img_cut)
        row_img.append(img_cut)
        #cv2.waitKey(1000)

    return row_img

pos_con = []
bitw_img = []

keep_day=[[],[],[],[],[]]

keep_box=[[],[],[],[],[]]


#read an image
img = cv2.imread("/content/drive/MyDrive/image_project/table1.jpg")

#resize image(optional)
"""scale_percent = 100
width = int(img.shape[1] * scale_percent / 100)
hight = int(img.shape[0] * scale_percent / 100)
dim = (width, hight)
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)"""

img_copy = img.copy()
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
threshold_img = cv2.adaptiveThreshold(gray_img, 255,
                                      cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,29,0)
dilation = cv2.dilate(threshold_img, (2, 2), 10)
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
contours = np.flip(contours, 0)
count = 0

if (len(contours) > 0):
    for i, con in enumerate(contours):
        areas_con = cv2.contourArea(con)
        x, y, w, h = cv2.boundingRect(con)
        if (areas_con < 19000 and areas_con > 4000):
                #print("index : {0} ==> area_con = {1}".format(i, areas_con))
                if (y>68) and (y<156):
                    keep_day[0].append(x)
                    keep_box[0].append((x, y, w, h))
                elif (y>156) and (y<244):
                    keep_day[1].append(x)
                    keep_box[1].append((x, y, w, h))
                elif (y>244) and (y<332):
                    keep_day[2].append(x)
                    keep_box[2].append((x, y, w, h))
                elif (y>332) and (y<420):
                    keep_day[3].append(x)
                    keep_box[3].append((x, y, w, h))
                elif (y>420) and (y<508):
                    keep_day[4].append(x)
                    keep_box[4].append((x, y, w, h))
                count += 1

    for row_day in keep_box:
          pos_con.append(box_cut(row_day))
    img_append = np.array(pos_con)

print("number of contours in condition is : {}".format(count))
cv2.waitKey(0)
cv2.destroyAllWindows()


number of contours in condition is : 50


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:232: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  m = asarray(m)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
np.append(pos_con, box_cut(keep_box[0]), axis=0)

In [ ]:
box_cut(keep_box[0])

In [46]:
img_append[0][0].shape[1]

94

In [55]:

for img in img_append[4]:
    print(img.shape[1])

94
98
100
100
100
98
99
196
99


In [ ]:
def extract(image):
    #closing
    kernel = np.ones((2,2),np.uint8)
    closing = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    # Apply OCR on the image
    costom_config=r'--oem 3 --psm '+str(6)
    text = pytesseract.image_to_string(closing, lang='tha', config= costom_config)
    return str(text)

# Mention the installed location of Tesseract-OCR in your system
pytesseract.pytesseract.tesseract_cmd = ( r'/usr/bin/tesseract' )
table=np.empty((0,12),dtype="<U8")
for j,row in enumerate(img_append):
    print(j)
    row_=np.empty(12,dtype="<U8")
    i = 0
    for img in row:
        text=extract(img)
        row_[i]=text
        i += 1
        if img.shape[1]>110: 
            row_[i]=text
            i += 1
    table = np.append(table, [row_], axis=0)
    print(j,'finish')


In [71]:
table

array([['จันทร์\n\x0c', '|   อ312', 'สุขพละ4/', ': ว31213', ': ว31213',
        '\x0c', 'อ31215 ส', '|     อ3', 'ว301116\n', '| ค30291',
        'ล:\n\x0c', '\x0c'],
       ['อังตาร\n\x0c', 'ค31203\n|', ' ค31105\n', 'ว31243\nท',
        'ว31243\nท', 'พัก\n\x0c', '|    ว30', 'โล\nวรณิน', 'โล\nวรณิน',
        'ว30282 ส', 'ว30282 ส', '\x0c'],
       ['๓\n\x0c', ', ค31105', 'แนะแนว -', '|       ', '|       ',
        '|       ', 'ส31 101\n', '| ท31101', 'ค31203\nว', 'อ30217  ',
        'อ30217  ', '\x0c'],
       ['พฤหัสบดี', ', ค31203', '| ห31101', '|  ศิลปะ', '|     ส3',
        '๒\n\x0c', 'อ31101\n ', '130201\nค', '    ชุมบ', 'ว30281 4',
        'ว30281 4', '\x0c'],
       ['ศุกร์\n\x0c', 'ว301117\n', '. ค31105', 'ว301118\n', 'ปทท่อททก',
        'พัก\n\x0c', ') คุณธรร', '|\nการงาน', '|\nการงาน', 'โ\n\x0c', '',
        '']], dtype='<U8')

In [9]:
for i in range(4,14):
    print(i)
    #try:
    # A text file is created and flushed
    try:
      file = open(f"output{str(i)}.txt", "x")
    except:
      file = open(f"output{str(i)}.txt", "w")
    file.write("")
    file.close()
    for img in img_append:

        # Mention the installed location of Tesseract-OCR in your system
        pytesseract.pytesseract.tesseract_cmd = ( r'/usr/bin/tesseract' )
        # Preprocessing the image starts

        # Convert the image to gray scale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Performing OTSU threshold
        ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)
        
        #closing
        kernel = np.ones((2,2),np.uint8)
        closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        
        file = open(f"output{str(i)}.txt", "a", encoding="utf-8")

        # Apply OCR on the image
        costom_config=r'--oem 3 --psm '+str(i)
        text = pytesseract.image_to_string(thresh1, lang='tha', config= costom_config)

        # Appending the text into file
        file.write(text)
        file.write("\n____________________\n")
        # Close the file
        file.close
    #except:
            #print('unsucessful')

4
5
6
7
8
9
10
11
12
13


In [19]:
thresh1

array([[255, 255, 255, ..., 255,   0,   0],
       [255, 255, 255, ..., 255, 255,   0],
       [255, 255, 255, ..., 255, 255,   0],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [  0,   0,   0, ..., 255, 255, 255]], dtype=uint8)